In [1]:
from pandas import DataFrame
from DATA225utils import make_connection, dataframe_query

In [2]:
def make_table(table, sql):
    cursor_warehouse.execute(f"DROP TABLE IF EXISTS {table}")
    cursor_warehouse.execute(sql)

In [3]:
def display_table(table, order_by=''):
    sql = f"SELECT * FROM {table}"
    
    if order_by != '':
        sql = sql + " ORDER BY " + order_by
        
    _, df = dataframe_query(cursor, sql)    
    return df

In [4]:
# conn = make_connection(config_file = 'Analytical_db.ini')
# cursor = conn.cursor()
conn = make_connection(config_file = 'techorcas_db.ini')
cursor = conn.cursor()

In [5]:
conn_warehouse = make_connection(config_file = 'techorcas_wh.ini')
cursor_warehouse = conn_warehouse.cursor()

## Date Dimension Table

In [20]:
#Dropping Churn_Customers table if already exists.
cursor_warehouse.execute("DROP TABLE IF EXISTS Date_Dimension")

In [21]:
sql = ( """
        CREATE TABLE Date_Dimension
        (   
           
            Date DATE ,
            Day varchar(10),
            Month varchar(10),
            Year int,
            Quarter varchar(10),
            PRIMARY KEY (DATE)
        )
        """
      )

cursor_warehouse.execute(sql)

In [22]:
#Insert vales into Churn_Customers
sql = ( """
        INSERT INTO Date_Dimension
        VALUES (%s, %s, %s, %s, %s )
        """
      )

In [23]:
def transform(row):
    if row[1] == 'NA':
        row[1] = 0


In [24]:
import csv
first = True
i = 0

with open(r'Date_Dimension_Table_Wh.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            transform(row)
            cursor_warehouse.execute(sql, row)

            
        first = False
    
conn_warehouse.commit()

## Product Dimension Table

In [25]:
#Dropping Churn_Customers table if already exists.
cursor_warehouse.execute("DROP TABLE IF EXISTS Product_Dimension")

In [26]:
sql = ( """
        CREATE TABLE Product_Dimension
        (   
            SKU VARCHAR(150),
            Style  VARCHAR(50),
            color VARCHAR(50),
            category VARCHAR(50),
            size VARCHAR(10),
            PRIMARY KEY (SKU)
        )
        """
      )

make_table('Product_Dimension', sql)

In [27]:
#Insert vales into Churn_Customers
sql = ( """
        INSERT INTO Product_Dimension
        VALUES (%s, %s, %s, %s, %s )
        """
      )

In [28]:
import csv
first = True
i = 0

with open(r'Product_Dimensional_Table.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            transform(row)
            cursor_warehouse.execute(sql, row)

            
        first = False
    
conn_warehouse.commit()

# 	Location Dimension Table

In [29]:
#Dropping Churn_Customers table if already exists.
cursor_warehouse.execute("DROP TABLE IF EXISTS Location_Dimension_Table")

In [30]:
sql = ( """
        CREATE TABLE Location_Dimension_Table
        (   
            ship_postal_code int,
            ship_country VARCHAR(50),
            ship_state VARCHAR(50),
            ship_city VARCHAR(50),
            PRIMARY KEY (ship_postal_code)
        )
        """
      )

make_table('Location_Dimension_Table', sql)


In [31]:
#Insert vales into Churn_Customers
sql = ( """
        INSERT INTO Location_Dimension_Table
        VALUES (%s, %s, %s, %s )
        """
      )

In [32]:
import csv
first = True
i = 0

with open(r'Location_Dimension_Table.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            transform(row)
            cursor_warehouse.execute(sql, row)

            
        first = False
    
conn_warehouse.commit()

## 	Fulfilment Dimension Table

In [33]:
#Dropping Churn_Customers table if already exists.
cursor_warehouse.execute("DROP TABLE IF EXISTS Fulfilment_Dimension_Table")

In [34]:
sql = ( """
        CREATE TABLE Fulfilment_Dimension_Table
        (   
Order_ID VARCHAR(150),
ship_service_level VARCHAR(150),
fulfilled_by VARCHAR(150),
Sales_Channel VARCHAR(150),
Fulfilment VARCHAR(150),
promotion_ids VARCHAR(3000),
Courier_Status VARCHAR(150),
status VARCHAR(150),

            PRIMARY KEY (Order_ID)
        )
        """
      )

make_table('Fulfilment_Dimension_Table', sql)


In [35]:
#Insert vales into Churn_Customers
sql = ( """
        INSERT INTO Fulfilment_Dimension_Table
        VALUES (%s, %s, %s, %s , %s, %s, %s, %s)
        """
      )

In [36]:
import csv
first = True
i = 0

with open(r'Fulfilment_Dimension_Table - Copy.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            transform(row)
            cursor_warehouse.execute(sql, row)

            
        first = False
    
conn_warehouse.commit()

## Sales Fact Table

In [37]:
#Dropping Churn_Customers table if already exists.
cursor_warehouse.execute("DROP TABLE IF EXISTS Sales_Fact_Table")

In [38]:
sql = ( """
        CREATE TABLE Sales_Fact_Table
        (   
Order_ID VARCHAR(150),
SKU VARCHAR(150),
Date DATE ,
Amount int,
B2B VARCHAR(50),
Inventory int NULL,
PRIMARY KEY (Order_ID,sku)
        )
        """
      )

make_table('Sales_Fact_Table', sql)


In [39]:
#Insert vales into Churn_Customers
sql = ( """
        INSERT INTO Sales_Fact_Table
        VALUES (%s, %s, %s, %s , %s, %s)
        """
      )

In [40]:
import csv
first = True
i = 0

with open(r'Sales_Fact_Table.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            transform(row)
            cursor_warehouse.execute(sql, row)

            
        first = False
    
conn_warehouse.commit()

## Shipping_Fact_Table

In [41]:

#Dropping Churn_Customers table if already exists.
cursor_warehouse.execute("DROP TABLE IF EXISTS Shipping_Fact_Table")

In [42]:
sql = ( """
        CREATE TABLE Shipping_Fact_Table
        (   
Order_ID VARCHAR(150),	
ship_postal_code int,	
Quantity int,

PRIMARY KEY (Order_ID)
        )
        """
      )

make_table('Shipping_Fact_Table', sql)


In [43]:
#Insert vales into Churn_Customers
sql = ( """
        INSERT INTO Shipping_Fact_Table
        VALUES (%s, %s, %s)
        """
      )

In [44]:
import csv
first = True
i = 0

with open(r'Shipping_Fact_Table.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            transform(row)
            cursor_warehouse.execute(sql, row)

            
        first = False
    
conn_warehouse.commit()

## Queries

In [ ]:
import mysql.connector
import pandas as pd

#### Query 1

In [ ]:
myresult = pd.read_sql('SELECT s.SKU, P.style, P.color, s.amount FROM Sales_Fact_Table s JOIN Product_Dimension P ON s.SKU = P.SKU',conn)

table=pd.DataFrame(myresult)
table=table.reset_index(drop=True)
pd.set_option('expand_frame_repr', False)
print(table)


#### Query 2

In [ ]:
myresult = pd.read_sql("SELECT d.day, d.month, d.year FROM Sales_Fact_Table s JOIN Date_Dimension d ON s.Date = d.Date where order_Id= '171-9918163-5396330';",conn)

table=pd.DataFrame(myresult)
table=table.reset_index(drop=True)
pd.set_option('expand_frame_repr', False)
print(table)

#### Query 3

In [ ]:
myresult = pd.read_sql('Select s.order_Id, f.fulfilment, f.Courier_Status, s.Quantity from shipping_fact_table s join Fulfilment_Dimension_Table f on s.order_id=f.order_id',conn)

table=pd.DataFrame(myresult)
table=table.reset_index(drop=True)
pd.set_option('expand_frame_repr', False)
print(table)

#### Query 4

In [ ]:
myresult = pd.read_sql('Select s.order_id , l.ship_postal_code, l.ship_country, l.ship_state, l.ship_city from shipping_fact_table s join location_dimension_table l on s.Ship_postal_code = l.ship_postal_code',conn)

table=pd.DataFrame(myresult)
table=table.reset_index(drop=True)
pd.set_option('expand_frame_repr', False)
print(table)

In [ ]:
cursor.close()
conn.close()